In [154]:
# multi headed multi-step cnn
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt

from query2 import *
from Func_Paper1 import *

In [155]:
df = data1()

df.index = df['datetime']
df.index = df.index.astype('datetime64[ns]')
df.sort_index(inplace=True)
df.dropna(inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'])
print("shape ", df.shape)

shape  (4410, 33)


In [156]:
df = df.resample('M').mean() 
print("shape ", df.shape)

shape  (198, 32)


In [157]:
df.dropna(inplace=True)
df.shape

(150, 32)

In [158]:
df = df.iloc[-18:,:]
print(df.shape)

(18, 32)


In [159]:
df.tail(1)

,tail24,tail24_avg,evap,infl,losses,rel1,rel2,rel3,rel_tol,engr1,...,run_c2,run_c3,spillway,irr,camp,demand,derate,outage,stor,fore24
datetime,,,,,,,,,,,,,,,,,,,,,
2020-06-30,84.324,82.234,0.5653,1.61998,1.15726,4.29272,4.58884,4.07944,12.961,588480.0,...,85.1,100.3,0.0,0.0,10423.4,12.8,73.8,0.0,4044.56128,139.7


In [160]:
now = df[['fore24']].iloc[-1:,:]
now.columns = ['stor30D']
now['datims'] = now.index
now = now[["datims", "stor30D"]]
now.reset_index(drop=True, inplace=True)

In [161]:
now

,datims,stor30D
0,2020-06-30,139.7


In [162]:
dfdate = pd.DataFrame(df.index)

In [163]:
dtat0 = dfdate.iloc[-1:,0:1].values[0][0]
dti = pd.date_range(dtat0, periods=6,freq='M').shift(1, freq='M')
dfdt = dti.to_frame(index=False, name='datims')

In [164]:
dfdt

,datims
0,2020-07-31
1,2020-08-31
2,2020-09-30
3,2020-10-31
4,2020-11-30
5,2020-12-31


In [165]:
# df.drop(('datetime'), axis=1, inplace=True)

In [166]:
from sklearn.preprocessing import MinMaxScaler
y_scaler = MinMaxScaler()
y_scaler.fit(df[['fore24']])
X_scaler = MinMaxScaler()

df[['tail24', 'tail24_avg', 'evap', 'infl', 'losses',
       'rel1', 'rel2', 'rel3', 'rel_tol', 'engr1', 'engr2', 'engr3', 'cond1',
       'cond2', 'cond3', 'str1', 'str2', 'str3', 'run_g1', 'run_g2', 'run_g3',
       'run_c1', 'run_c2', 'run_c3', 'spillway', 'irr', 'camp', 'demand',
       'derate', 'outage', 'stor','fore24']] = X_scaler.fit_transform(df)

dfval = df.values[:,:-1]

In [167]:
def split_dataset(data):
    test = data
    test = array(split(test, len(test)/6))
    return test

In [168]:
test = split_dataset(df.values)

In [169]:
print('test', test.shape)

test (3, 6, 32)


In [170]:
n_input = 14

In [171]:
def forecast(model, history, n_input):

    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))

    input_x = data[-n_input:, :]

    input_x = [input_x[:,i].reshape((1,input_x.shape[0],1)) for i in range(input_x.shape[1])]

    yhat = model.predict(input_x, verbose=1)

    yhat = yhat[0]
    
    return yhat

In [172]:
#เป็น list ที่สมาชิกแต่ละตัวเป็น samples (windows, n-feature)
history = [x for x in test]

In [173]:
print(len(history))
print(type(history))
print((array(history).shape))
print(type(array(history)))

3
<class 'list'>
(3, 6, 32)
<class 'numpy.ndarray'>


In [174]:
model = ShallowConvNet(test, n_input)
# model.summary()
model.load_weights("models/weights04.hdf5")
model.compile(loss='mse', optimizer='adam')

In [175]:
# model.summary()

In [176]:
predictions = list()
for i in range(len(test)):
    # predict the week
    yhat_sequence = forecast(model, history, n_input)
    # store the predictions
    predictions.append(yhat_sequence)
predictions = array(predictions)

1/1 [==============================] - 0s 15ms/step


In [177]:
predictions[0].reshape(-1,1).shape

(6, 1)

In [178]:
yp = y_scaler.inverse_transform(predictions[0].reshape(1,-1))

In [179]:
yp

array([[142.83452, 146.84393, 151.17636, 152.4822 , 155.09238, 152.2679 ]],
      dtype=float32)

In [180]:
from sqlalchemy import create_engine
user='root'
pwd='opt#565784'
dbname='predictive'
host='10.211.1.25'
port=3306
con = create_engine(f'mysql+pymysql://{user}:{pwd}@{host}:{port}/{dbname}')


In [181]:

df = pd.DataFrame(yp).T
df.columns = ['stor30D']
com = pd.concat([dfdt, df], axis=1, join='inner')


In [182]:
com

,datims,stor30D
0,2020-07-31,142.834518
1,2020-08-31,146.843933
2,2020-09-30,151.176361
3,2020-10-31,152.482193
4,2020-11-30,155.092377
5,2020-12-31,152.267899


In [183]:
com = now.append(com) 
com.reset_index(drop=True, inplace=True)

In [184]:
com

,datims,stor30D
0,2020-06-30,139.700000
1,2020-07-31,142.834518
2,2020-08-31,146.843933
3,2020-09-30,151.176361
4,2020-10-31,152.482193
5,2020-11-30,155.092377
6,2020-12-31,152.267899


In [185]:
com.to_sql('yearly_m1', con, if_exists='append', index=False)